In [1]:
# 本代码构建了AlexNet卷积神经网络，并使用CIFAR10数据集进行训练完成识别任务

In [52]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms

In [53]:
# 运算设备设置为显卡
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [63]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整到AlexNet需要的尺寸
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = torchvision.datasets.CIFAR10('data', 
                                        train=True, 
                                        transform=transform, 
                                        download=True)
test_set = torchvision.datasets.CIFAR10('data', 
                                       train=False, 
                                       transform=transform, 
                                       download=True)

100%|████████████████████████████████████████████████████████████████████████████████| 170M/170M [03:32<00:00, 803kB/s]


In [64]:
# 设置批次大小
BATCH_SIZE= 256

In [65]:
# 加载训练集和测试集
train_loader = data.DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(test_set, batch_size=BATCH_SIZE)

In [76]:
# 定义AlexNet网络架构
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        # 卷积部分
        self.c1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.relu = nn.ReLU()
        self.s2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.c3 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.s4 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.c5 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.c6 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.c7 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.s8 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6))
        self.flatten = nn.Flatten()
        # 全连接部分
        self.f1 = nn.Linear(256 * 6 * 6, 4096)
        self.dropout1 = nn.Dropout(0.5)
        self.f2 = nn.Linear(4096, 4096)
        self.dropout2 = nn.Dropout(0.5)
        self.f3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.relu(self.c1(x))
        x = self.s2(x)
        x = self.relu(self.c3(x))
        x = self.s4(x)
        x = self.relu(self.c5(x))
        x = self.relu(self.c6(x))
        x = self.relu(self.c7(x))
        x = self.s8(x)
        x = self.adaptive_pool(x)
        x = self.flatten(x)
        x = self.relu(self.f1(x))
        x = self.dropout1(x)
        x = self.relu(self.f2(x))
        x = self.dropout2(x)
        x = self.f3(x)
        return x

In [77]:
# 将模型移送到GPU内存中
model = AlexNet()
model.to(device)

AlexNet(
  (c1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (relu): ReLU()
  (s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (s4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c5): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c6): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c7): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (s8): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adaptive_pool): AdaptiveAvgPool2d(output_size=(6, 6))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (f1): Linear(in_features=9216, out_features=4096, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (f2): Linear(in_features=4096, out_features=4096, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (f3): Linear(in_features=409

In [86]:
# 设置损失函数
loss_func = nn.CrossEntropyLoss()
# 设置学习率
lr = 0.01
# 设置优化方法
optimizer = torch.optim.Adam(model.parameters(), lr)

In [87]:
# 设定训练轮次
EPOCH = 5
# 记录训练准确度的历史
train_acc_history = []

In [ ]:
# 模型训练
for epoch in range(EPOCH):
    training_loss = 0.0
    training_acc = 0.0

    model.train()
    for batch, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_func(pred, y)

        # 清除梯度，否则会将每一个batch的梯度累加
        optimizer.zero_grad()
        # 反向传播(计算梯度)
        loss.backward()
        # 更新位置参数的值
        optimizer.step()

        training_loss += loss.item()
        training_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
    training_acc /= len(train_loader.dataset)
    training_loss /= len(train_loader)

    train_acc_history.append(training_acc)  # 记录训练准确度
    print(f'Epoch {epoch + 1}: training accuracy: {training_acc:4f}, training loss: {training_loss:4f}')

In [ ]:
# 模型测试
test_loss = 0.0
test_acc = 0.0
for batch, (X, y) in enumerate(test_loader):
    X = X.to(device)
    y = y.to(device)
    pred = model(X)
    loss = loss_func(pred, y)
    test_loss += loss.item()
    test_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
test_acc /= len(test_loader.dataset)
test_loss /= len(test_loader)
print(f'test accuracy: {test_acc:4f}', end=', ')
print(f'test loss: {test_loss:4f}')

# 绘制训练准确度随轮次变化的图
# plt.plot(range(1, EPOCH+1), train_acc_history, marker='o', color='b', label='Max Pooling - Accuracy')
plt.plot(range(1, EPOCH+1), train_acc_history, marker='o', color='g', label='Avg Pooling - Accuracy')
# plt.plot(range(1, EPOCH+1), train_acc_history, marker='o', color='r', label='Min Pooling - Accuracy')

plt.title('Training Accuracy over Epochs with Avg Pooling Methods')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()